In [0]:
# CRM Data Ingestion (Batch)
crm_df = spark.table("enterprise_modernization.default.crm_data")
crm_df.show(5)

# Save to Bronze table (optionally overwrite each run)
crm_df.write.mode("overwrite").saveAsTable("enterprise_modernization.bronze.bronze_crm")

In [0]:
df = spark.table("enterprise_modernization.bronze.bronze_crm")

print(df.columns)

print(df.count())

df.show(5)